In [2]:
# RAG Pipeline (Generic, Korean-aware) - dev_center_guide_allmd_touched.md
# - Embeddings: bge-m3:latest (Ollama)
# - LLM: exaone3.5:latest (Ollama)
# - Hybrid retrieval: FAISS + BM25
# - Query normalization: Korean particles split, camelCase split
# - Metadata: header path, tags (acronyms, camel/snake case, http codes, etc.)
# - Rerank: keyword/metadata-based precision rerank
# - Table parsing: Markdown table rows -> structured docs priority

import os, re, json
from typing import List, Dict, Any, Tuple, Optional, Set
from dataclasses import dataclass
from collections import Counter

from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

try:
    from langchain_text_splitters import MarkdownHeaderTextSplitter
    HAS_MD_SPLITTER = True
except Exception:
    HAS_MD_SPLITTER = False

DATA_FILE = "data/dev_center_guide_allmd_touched.md"
EMBED_MODEL = "bge-m3:latest"
LLM_MODEL = "exaone3.5:latest"
# LLM_MODEL = "mistral:latest"
FAISS_DIR = "models/faiss_rag_base_bge-m3_generic_3"

print("Config:")
print(f"- DATA_FILE: {DATA_FILE}")
print(f"- EMBED_MODEL: {EMBED_MODEL}")
print(f"- LLM_MODEL: {LLM_MODEL}")
print(f"- FAISS_DIR:  {FAISS_DIR}")

Config:
- DATA_FILE: data/dev_center_guide_allmd_touched.md
- EMBED_MODEL: bge-m3:latest
- LLM_MODEL: exaone3.5:latest
- FAISS_DIR:  models/faiss_rag_base_bge-m3_generic_3


In [3]:
# Query Normalizer
import re
class QueryNormalizer:
    def __init__(self):
        self.korean_particles = [
            '의','는','은','이','가','를','을','에','와','과','도','로','으로',
            '부터','까지','만','라도','께서','에서','에게','한테','보다','처럼','같이','마저','조차','뿐'
        ]
        self.camel_split = re.compile(r'([a-z])([A-Z])')
        self.acronym_pattern = re.compile(r'[A-Z0-9_]{2,}')

    def separate_korean_particles(self, text: str) -> str:
        for p in self.korean_particles:
            text = re.sub(rf'([A-Za-z0-9_]+){re.escape(p)}', r'\1 ' + p, text)
        return text

    def split_camel(self, text: str) -> str:
        return self.camel_split.sub(r'\1 \2', text)

    def normalize(self, text: str) -> str:
        t = self.separate_korean_particles(text)
        t = self.split_camel(t)
        t = re.sub(r'\s+', ' ', t).strip()
        return t

    def keywords_from_query(self, text: str) -> list:
        t = self.normalize(text)
        tokens = set()
        tokens.update(self.acronym_pattern.findall(t))
        for w in t.split():
            if re.match(r'[A-Za-z0-9_]{3,}', w):
                tokens.add(w)
        return list(tokens)

In [4]:
# Tag Extractor
import re, json
class TechnicalTermExtractor:
    def __init__(self):
        self.patterns = {
            'camel_case': re.compile(r'\b[a-z]+[A-Z][a-zA-Z0-9]*\b'),
            'snake_case': re.compile(r'\b[a-z]+_[a-z0-9_]+\b'),
            'all_caps_codes': re.compile(r'\b[A-Z][A-Z0-9_]{1,}\b'),
            'compound_codes': re.compile(r'\b[A-Z]{3,}(?:[A-Z0-9_]*[A-Z0-9]+)+\b'),
            'long_caps_words': re.compile(r'\b[A-Z]{6,}\b'),
            'acronyms': re.compile(r'\b[A-Z]{2,5}\b'),
            'code_values': re.compile(r'\b[A-Z]+_[A-Z0-9]+\b|\b\d+\.\d+\.\d+[A-Z]?\b|\b[A-Z]{2,}_[0-9]{6,}\b'),
            'http_terms': re.compile(r'\b(?:GET|POST|PUT|DELETE|PATCH|HEAD|OPTIONS)\b|\b\d{3}\s+[A-Z]+\b'),
            'json_fields': re.compile(r'"([a-zA-Z][a-zA-Z0-9_]*)"'),
            'table_params': re.compile(r'\|\s*([a-zA-Z][a-zA-Z0-9_]*)\s*\|'),
            'table_codes': re.compile(r'\|\s*([A-Z][A-Z0-9_]{2,})\s*\|'),
            'numeric_codes': re.compile(r'\b[A-Z]*[0-9]+[A-Z]*\b|\b[0-9]+[A-Z]+\b'),
            'mixed_case': re.compile(r'\b[a-zA-Z]*[a-z][A-Z][a-zA-Z0-9]*\b'),
        }
        self.exclude_words = {
            'THE','AND','OR','BUT','FOR','WITH','FROM','TO','IN','ON','AT','BY','AS','IS','ARE','WAS','WERE','BE','BEEN',
            'HAVE','HAS','HAD','DO','DOES','DID','WILL','WOULD','COULD','SHOULD','MAY','MIGHT','CAN','MUST','SHALL',
            'THIS','THAT','THESE','THOSE','A','AN','IF','WHEN','WHERE','HOW','WHY','WHAT','WHO','WHICH','WHOSE','WHOM',
            'ALL','ANY','SOME','MANY','MUCH','MORE','MOST','OTHER','ANOTHER','EACH','EVERY','BOTH','EITHER','NEITHER',
            'NOT','NO','YES','String','ID','URL','URI','JSON','XML','HTTP','HTTPS','SSL','TLS','UTF','ASCII','Base64'
        }

    def _is_valid_tag(self, tag: str) -> bool:
        if len(tag) < 2 or len(tag) > 40: return False
        if tag in self.exclude_words: return False
        if tag.isdigit() and len(tag) < 3: return False
        if re.match(r'^[^a-zA-Z0-9]+$', tag): return False
        return True

    def extract(self, text: str) -> list:
        tags = set()
        for _, pattern in self.patterns.items():
            matches = pattern.findall(text)
            for m in matches:
                tag = m if isinstance(m, str) else m[0]
                if self._is_valid_tag(tag):
                    tags.add(tag)
        return list(tags)

In [5]:
# Load markdown + split + tables
import re, json
from langchain.docstore.document import Document

def load_markdown(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def parse_markdown_tables(md_text: str) -> list:
    docs = []
    table_blocks = re.findall(r'((?:^\|.*\|\s*\n)+)', md_text, flags=re.MULTILINE)
    for tb in table_blocks:
        rows = [r.strip() for r in tb.strip().splitlines() if r.strip().startswith('|')]
        if len(rows) < 2: continue
        header = [c.strip() for c in rows[0].strip('|').split('|')]
        data_rows = []
        for r in rows[1:]:
            if re.match(r'^\|\s*:?-+:?\s*(\|\s*:?-+:?\s*)+\|?$', r):
                continue
            data_rows.append([c.strip() for c in r.strip('|').split('|')])
        for dr in data_rows:
            content = {}
            for i, h in enumerate(header):
                if i < len(dr): content[h] = dr[i]
            text = json.dumps(content, ensure_ascii=False)
            docs.append(Document(page_content=text, metadata={'type': 'table_row', 'headers': header}))
    return docs

def split_markdown_to_docs(md_text: str) -> list:
    docs = []
    if HAS_MD_SPLITTER:
        splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
            ("#", "h1"), ("##","h2"), ("###","h3"), ("####","h4")
        ])
        splits = splitter.split_text(md_text)
        for s in splits:
            path = []
            for k in ['h1','h2','h3','h4']:
                if s.metadata.get(k):
                    path.append(s.metadata[k])
            full_path = " > ".join(path) if path else ""
            content = (f"[{full_path}] " if full_path else "") + s.page_content
            docs.append(Document(page_content=content, metadata={'full_path': path, 'type': 'section'}))
    else:
        parts = re.split(r'(?m)^#{1,4}\s+', md_text)
        for p in parts:
            p = p.strip()
            if not p: continue
            first_line = p.splitlines()[0] if '\n' in p else p
            content = f"[{first_line}] " + p
            docs.append(Document(page_content=content, metadata={'full_path': [first_line], 'type': 'section'}))
    return docs

In [6]:
##### ONLY FOR TESTING #####
md_text = load_markdown(DATA_FILE)
base_docs = split_markdown_to_docs(md_text)

cnt = 0
for doc in base_docs:
  if 'PNS' in doc.page_content:
    print(doc.metadata)
    print(f"++++++ idx: {cnt} ++++++")
    print(doc.page_content)
    print("-" * 100)
    cnt += 1

print(f"cnt: {cnt}")

{'full_path': ['원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드'], 'type': 'section'}
++++++ idx: 0 ++++++
[원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드] 원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download](v21/sample

In [8]:
# Build docs with tags
from langchain.docstore.document import Document

def build_documents(md_text: str) -> list[Document]:
    base_docs = split_markdown_to_docs(md_text)
    table_docs = parse_markdown_tables(md_text)
    tte = TechnicalTermExtractor()

    enriched_docs = []
    for d in base_docs:
        tags = tte.extract(d.page_content)
        md = dict(d.metadata)
        md['tags'] = tags
        enriched_docs.append(Document(page_content=d.page_content, metadata=md))

    enriched_tables = []
    for td in table_docs:
        md = dict(td.metadata)
        md['tags'] = list(TechnicalTermExtractor().extract(td.page_content))
        md['priority'] = 'table'
        enriched_tables.append(Document(page_content=td.page_content, metadata=md))

    all_docs = enriched_docs + enriched_tables
    print(f"Docs: sections={len(enriched_docs)}, table_rows={len(enriched_tables)}, total={len(all_docs)}")
    return all_docs

md_text = load_markdown(DATA_FILE)
docs = build_documents(md_text)

Docs: sections=495, table_rows=518, total=1013


In [10]:
###### TEST ######
cnt = 0
for doc in docs:
    if 'PNS' in doc.page_content:
        cnt += 1
        print(doc.metadata)
        print("--> tags :", doc.metadata['tags'])
        print("++++++")
        print(doc.page_content)
        print("-" * 100)
## {'full_path': ['원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드'], 'type': 'section', 'tags': ['V4', 'V7', 'info', 'SDK', 'PNS', 'IAP', 'V21', 'API', 'V16']}

{'full_path': ['원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드'], 'type': 'section', 'tags': ['IAP', 'V4', 'V16', 'V7', 'API', 'PNS', 'info', 'V21', 'SDK']}
--> tags : ['IAP', 'V4', 'V16', 'V7', 'API', 'PNS', 'info', 'V21', 'SDK']
++++++
[원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드] 원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.  
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
{% hint style="info" %}
API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 [여기](old-version/v16)를 클릭해주세요.
{% endhint %}  
{% hint style="info" %}
현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요  
* [대한민국 외 국가 및 지역 배포를 위한 가이드](../glb)
{% endhint %}  
If you are comfortable with English, please change the language to English from the upper left side in this page.  
* [01. 원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification S

In [11]:
# Build retrievers
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

embeddings = OllamaEmbeddings(model=EMBED_MODEL)

if os.path.isdir(FAISS_DIR):
    try:
        vs = FAISS.load_local(FAISS_DIR, embeddings, allow_dangerous_deserialization=True)
        print("FAISS loaded:", FAISS_DIR)
    except Exception as e:
        print("FAISS load failed, rebuilding:", e)
        vs = FAISS.from_documents(docs, embeddings)
        vs.save_local(FAISS_DIR)
        print("FAISS built and saved.")
else:
    vs = FAISS.from_documents(docs, embeddings)
    vs.save_local(FAISS_DIR)
    print("FAISS built and saved:", FAISS_DIR)

vector_retriever = vs.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 20, "fetch_k": 60, "lambda_mult": 0.7}
)

bm25 = BM25Retriever.from_documents(docs, bm25_params={"k1": 1.5, "b": 0.75})
bm25.k = 30

ensemble = EnsembleRetriever(
    retrievers=[bm25, vector_retriever],
    weights=[0.4, 0.6]
)
print("Retrievers ready.")

FAISS loaded: models/faiss_rag_base_bge-m3_generic_3
Retrievers ready.


In [12]:
bm25 = BM25Retriever.from_documents(docs, bm25_params={"k1": 1.5, "b": 0.75})
bm25.k = 30

res_docs = bm25.invoke("PNS 의 purchaseState 값은 무엇인가요?")
cnt = 0
for doc in res_docs:
    cnt += 1
    print(doc.page_content)
    print(cnt, "-" * 100)
    
print(f"cnt: {cnt}")

[PurchaseClient.ConnectionState] ```
public static @interface PurchaseClient.ConnectionState
```  
```
com.gaa.sdk.iap.PurchaseClient.ConnectionState
```  
PurchaseClient 의 연결 상태 값.
1 ----------------------------------------------------------------------------------------------------
{"구분": "변경", "API 목록": "월정액 상품 구매 상세조회", "API URI - V5(SDK V17)": "/v2/purchase/recurring-details/{purchaseId}/{packageName}", "API URI - V6(SDK V19)": "<p>/v6/apps/{packageName}/purchases/auto/products<br>/{productId}/{purchaseToken}</p>", "변경사항": "<p>응답규격에 acknowledgeState, lastPurchaseId, lastPurchaseState 추가</p><p>응답규격에 price, developerPayload, purchaseState 삭제</p>"}
2 ----------------------------------------------------------------------------------------------------
[PurchaseClient > Public methods <a href="#id-c-purchaseclient-publicmethods" id="id-c-purchaseclient-publicmethods"></a> > getConnectionState <a href="#id-c-purchaseclient-getconnectionstate" id="id-c-purchaseclient-getconnectionstate"><

In [13]:
### ONLY FOR TESTING ####
qry = "PNS의 purchaseState 값은 무엇인가요?"
res = ensemble.invoke(qry)
for r in res:
    print(r.page_content)
    print("-" * 100)
    
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_ollama import ChatOllama


# LLM 초기화
llm = ChatOllama(model="exaone3.5:latest", temperature=0.1)

# LLMChainExtractor 사용
compressor = LLMChainExtractor.from_llm(llm)

# 문맥 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble
)

res = compression_retriever.invoke(
    "PNS 메세지의 purchaseState 값은 무엇인가요?"
)

print("=" * 100)
cnt = 0
for doc in res:
    print(f"--> idx: {cnt}")
    print(doc.page_content)
    print("-" * 100)
    cnt += 1


[07. PNS(Payment Notification Service) 이용하기 > **개요**] PNS는 Payment Notification Service의 약자입니다. PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 원스토어의 서버가 개별 사용자의 결제 상태(결제 완료, 결제 취소)를 메시지로 전송하여 결제 트랜젝션의 상태를 손실없이 알려주기 위한 용도의 기능입니다. 즉, 개발사가 지정한 서버에서 원스토어가 정의한 규칙에 맞추어 API를 구현하면 해당 API를 원스토어의 결제 담당 서버에서 호출하는 형태입니다.  
Server to Server, 즉 서버간에 데이터를 전송한다고 할지라도 네트워크 문제로 메세지 전송 실패가 발생하기 때문에 200 OK로 응답을 인지하지 못할 경우 반복하여 메시지가 전송될 수 있습니다. 개발사의 서버는 메시지를 수신후 정의된 응답을 하여야 원스토어는 개발사 서버가 정상적으로 메시지를 전달 받았음을 인지합니다.  
결제 트렌젝션의 유형에 따라 아래와 같은 유형의 메세지가 전송됩니다.  
* 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service)&#x20;
* 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service)&#x20;  
{% hint style="warning" %}
Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로, notification 수신을 기준으로 상품(서비스)을 제공하는 것은 권장하지 않습니다.&#x20;  
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.  
원스토어는 검증 및 모니터링 목적으로 결제 테스트를 진행할 수 있으며, 해당 테스트 건들도 결제/결제취소 시 동일하

In [15]:
# Precision rerank + smart retrieval
from langchain.docstore.document import Document

normalizer = QueryNormalizer()

def generate_query_variants(query: str) -> list:
    base = query
    norm = normalizer.normalize(query)
    variants = [base]
    if norm != base:
        variants.append(norm)
    kws = normalizer.keywords_from_query(query)
    if kws:
        variants.append(" ".join(kws))
    return list(dict.fromkeys(variants))

def precision_rerank(docs: list, query: str, top_k: int = 10) -> list:
    norm = normalizer.normalize(query)
    tokens = set(normalizer.keywords_from_query(query))
    primary = [t for t in tokens if re.match(r'^[A-Z0-9_]{2,}$', t)]
    secondary = [t for t in tokens if t not in primary]
    if 'purchase' in norm.lower():
        secondary.extend(['purchaseState', 'purchase_state'])

    scored = []
    for d in docs:
        txt = d.page_content
        up = txt.upper()
        score = 0
        for kw in primary:
            if kw.upper() in up:
                score += 10 + up.count(kw.upper()) * 2
        for kw in secondary:
            if kw.lower() in txt.lower():
                score += 3
        md = d.metadata or {}
        tags = set((md.get('tags') or []))
        if tags and any(k in tags for k in primary):
            score += 4
        if md.get('type') == 'table_row':
            score += 6
        if isinstance(md.get('full_path'), list):
            path_str = " ".join(md['full_path'])
            for kw in primary:
                if kw.lower() in path_str.lower():
                    score += 2
        scored.append((score, d))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [d for s, d in scored[:top_k]]

def smart_retrieve(query: str, top_k: int = 10) -> list:
    vars_ = generate_query_variants(query)
    pool = []
    seen = set()
    for v in vars_:
        try:
            res = ensemble.invoke(v)
        except Exception:
            res = []
        for d in res:
            key = hash(d.page_content[:256])
            if key not in seen:
                seen.add(key)
                pool.append(d)
        if len(pool) >= top_k * 3:
            break
    if not pool:
        return []
    return precision_rerank(pool, query, top_k=top_k)

In [16]:
### ONLY FOR TESTING ####
qry = "PNS의 purchaseState 값은 무엇인가요?"
res = generate_query_variants(qry)
for r in res:
    print(r)
    print("-" * 100)

# res = smart_retrieve("PNS의 purchaseState 값은 무엇인가요?", top_k=10)
# res = smart_retrieve("PNS의 개념은 무엇인가요?", top_k=10)

# idx = 0
# for doc in res:
#     print(f"--> idx: {idx}")
#     print(doc.page_content)
#     print("-" * 100)
#     idx += 1

PNS의 purchaseState 값은 무엇인가요?
----------------------------------------------------------------------------------------------------
PNS 의 purchase State 값은 무엇인가요?
----------------------------------------------------------------------------------------------------
purchase State PNS
----------------------------------------------------------------------------------------------------


In [17]:
# LLM chain (exaone3.5:latest)
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate.from_template(
    """당신은 기술 문서 기반 Q/A 어시스턴트입니다.
아래 컨텍스트를 근거로 질문에 답하세요. 추측하지 마세요.
가능하면 표/필드값을 그대로 근거로 답하고, 출처 섹션/헤더를 함께 언급하세요.
한국어로 답변해주세요.

질문: {question}

컨텍스트:
{context}

요약 답변:"""
)

llm = ChatOllama(model=LLM_MODEL)
parser = StrOutputParser()

def format_context(docs: list) -> str:
    parts = []
    for i, d in enumerate(docs, 1):
        md = d.metadata or {}
        path = " > ".join(md.get('full_path', [])) if isinstance(md.get('full_path'), list) else ""
        prefix = f"[{path}] " if path else ""
        text = d.page_content
        if md.get('type') == 'table_row':
            text = "(표 행) " + text
        parts.append(f"({i}) {prefix}{text}")
    return "\n\n".join(parts)

def answer(query: str, top_k: int = 7):
    docs = smart_retrieve(query, top_k=top_k)
    ctx = format_context(docs)
    chain = prompt_template | llm | parser
    output = chain.invoke({"question": query, "context": ctx})
    return output, docs

/tmp/ipykernel_5434/599850452.py:20: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=LLM_MODEL)


In [19]:
# Quick test
tests = [
    # "PNS의 purchaseState 값은 무엇인가요?", 
    "PNS 메세지 요소중 'purchaseState'의 코드값은 무엇인가요?", 
    # "PNS에서 발송되는 메시지 Example를 알려주세요"
]

for q in tests:
    print("="*80)
    print("Q:", q)
    out, src = answer(q, top_k=4)
    print("\n[Answer]\n", out)
    print("\n[Top sources]")
    for i, d in enumerate(src, 1):
        md = d.metadata or {}
        path = " > ".join(md.get('full_path', [])) if isinstance(md.get('full_path'), list) else ""
        print(f"- {i}) type={md.get('type')} path={path} tags_sample={(md.get('tags') or [])[:100]}")

Q: PNS 메세지 요소중 'purchaseState'의 코드값은 무엇인가요?

[Answer]
 'purchaseState' 요소의 코드값은 **"COMPLETED"** 및 **"CANCELED"** 입니다. 이 값들은 결제 상태를 나타내며, 각각 결제 완료와 결제 취소를 의미합니다. 이 정보는 [PNS Payment Notification 메시지 발송 규격](07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버)) 섹션에서 확인 가능합니다.

[Top sources]
- 1) type=section path=07. PNS(Payment Notification Service) 이용하기 > **개요** tags_sample=['200', 'warning', 'SNS', 'PNS']
- 2) type=section path=07. PNS(Payment Notification Service) 이용하기 > **PNS 상세** > PNS Payment Notification 메시지 발송 규격 (원스토어 → 개발사 서버) > **Signature 검증 방법** tags_sample=['0900001234', 'END', 'initVerify', 'xSd9', 'SIGN_ALGORITHM', 'signature', 'JSON_UNESCAPED_UNICODE', 'getInstance', 'readTree', 'SINGLE_PAYMENT_TRANSACTION', 'unverified', 'BEGIN', 'yYWpUr7om', 'getValueAsText', 'SHA512withRSA', 'wrap', 'PNS', 'formatSignature', 'openssl_verify', 'MNxIl32ws', 'ensure_ascii', 'ONESTORECASH', '3.1.0D', 'fmyV6', 'Python', 'productName', 

In [15]:
### ONLY FOR TESTING ####

# 개선된 표 파서 - 필드-값 매핑 강화
def enhanced_table_parser(md_text: str) -> list:
    docs = []
    
    # 마크다운 표에서 | 필드명 | 타입 | 설명 | 형태의 패턴 찾기
    table_pattern = r'\|\s*([A-Za-z][A-Za-z0-9_]*)\s*\|\s*[^|]*\s*\|\s*([^|]+)\s*\|'
    
    for match in re.finditer(table_pattern, md_text):
        field_name = match.group(1).strip()
        description = match.group(2).strip()
        
        # 특히 "필드명 : 값1 / 값2" 패턴 감지
        if ':' in description and '/' in description:
            values = [v.strip() for v in description.split('/') if ':' in v or v.strip().isupper()]
            if values:
                structured_doc = {
                    "field": field_name,
                    "description": description,
                    "possible_values": values
                }
                content = f"필드: {field_name}\n설명: {description}\n가능한 값들: {', '.join(values)}"
                docs.append(Document(
                    page_content=content,
                    metadata={
                        'type': 'field_definition',
                        'field': field_name,
                        'priority': 'high',
                        'tags': [field_name] + [v.split(':')[0].strip() for v in values if ':' in v]
                    }
                ))
    
    return docs
  
# 강화된 정밀 재랭킹 - 필드값 정의 우선순위
def enhanced_precision_rerank(docs: list, query: str, top_k: int = 10) -> list:
    norm = normalizer.normalize(query)
    tokens = set(normalizer.keywords_from_query(query))
    
    # 쿼리에서 "필드의 값" 패턴 감지
    field_value_pattern = r'([A-Za-z][A-Za-z0-9_]+)\s*의\s*([A-Za-z][A-Za-z0-9_]+)\s*값'
    field_match = re.search(field_value_pattern, query)
    
    primary = [t for t in tokens if re.match(r'^[A-Z0-9_]{2,}$', t)]
    secondary = [t for t in tokens if t not in primary]
    
    if field_match:
        context_term = field_match.group(1)  # e.g., "PNS"
        field_term = field_match.group(2)    # e.g., "purchaseState"
        primary.extend([context_term, field_term])
        print(f"🎯 필드값 쿼리 감지: {context_term} -> {field_term}")

    scored = []
    for d in docs:
        txt = d.page_content
        up = txt.upper()
        score = 0
        
        # 기본 키워드 매칭
        for kw in primary:
            if kw.upper() in up:
                score += 10 + up.count(kw.upper()) * 2
        
        for kw in secondary:
            if kw.lower() in txt.lower():
                score += 3
        
        md = d.metadata or {}
        
        # 필드 정의 타입 대폭 가산점
        if md.get('type') == 'field_definition':
            score += 15
            if field_match and field_match.group(2).lower() == md.get('field', '').lower():
                score += 20  # 정확한 필드 매칭시 추가 점수
        
        # 표 데이터 가산점
        if md.get('type') == 'table_row':
            score += 8
        
        # 우선순위 가산점
        if md.get('priority') == 'high':
            score += 5
        
        # 태그 매칭 가산점
        tags = set(md.get('tags', []))
        if tags and any(k.upper() in [t.upper() for t in tags] for k in primary):
            score += 6
        
        # 헤더 경로 매칭
        if isinstance(md.get('full_path'), list):
            path_str = " ".join(md['full_path']).upper()
            for kw in primary:
                if kw.upper() in path_str:
                    score += 3
        
        scored.append((score, d))
    
    scored.sort(key=lambda x: x[0], reverse=True)
    
    # 디버그 정보
    print(f"�� 상위 3개 문서 점수:")
    for i, (score, doc) in enumerate(scored[:3]):
        md = doc.metadata or {}
        print(f"  {i+1}. 점수:{score} | 타입:{md.get('type')} | 필드:{md.get('field')}")
    
    return [d for s, d in scored[:top_k]]
  
# 강화된 문서 빌더 - 필드 정의 포함
def build_enhanced_documents(md_text: str) -> list:
    base_docs = split_markdown_to_docs(md_text)
    table_docs = parse_markdown_tables(md_text)
    field_docs = enhanced_table_parser(md_text)  # 새로운 필드 정의 파서
    
    tte = TechnicalTermExtractor()

    # 기존 섹션 문서 강화
    enriched_docs = []
    for d in base_docs:
        tags = tte.extract(d.page_content)
        md = dict(d.metadata)
        md['tags'] = tags
        enriched_docs.append(Document(page_content=d.page_content, metadata=md))

    # 기존 표 데이터
    enriched_tables = []
    for td in table_docs:
        md = dict(td.metadata)
        md['tags'] = list(TechnicalTermExtractor().extract(td.page_content))
        md['priority'] = 'table'
        enriched_tables.append(Document(page_content=td.page_content, metadata=md))

    all_docs = enriched_docs + enriched_tables + field_docs
    print(f"📊 문서 통계: 섹션={len(enriched_docs)}, 표={len(enriched_tables)}, 필드정의={len(field_docs)}, 총={len(all_docs)}")
    return all_docs

# 기존 docs 업데이트
enhanced_docs = build_enhanced_documents(md_text)
print(f"✅ 강화된 문서 세트가 준비되었습니다!")

# 강화된 LLM 프롬프트 - 필드값 추출 특화
enhanced_prompt_template = PromptTemplate.from_template(
    """당신은 기술 문서 기반 Q/A 어시스턴트입니다.
아래 컨텍스트를 근거로 질문에 답하세요.

**중요 지시사항:**
1. 특정 필드의 값이나 코드를 묻는 질문인 경우, 정확한 값들을 나열하세요
2. "필드: xxx" 로 시작하는 정보가 있으면 우선적으로 참고하세요
3. 표 형태의 정보에서 "값1 / 값2" 또는 "값1 : 설명1 / 값2 : 설명2" 패턴을 찾아 정확히 추출하세요
4. 추측하지 말고 문서에 명시된 내용만 답변하세요

질문: {question}

컨텍스트:
{context}

답변 (가능한 값들을 명확히 나열):"""
)

def enhanced_answer(query: str, top_k: int = 8):
    # 강화된 검색 및 재랭킹 사용
    vars_ = generate_query_variants(query)
    pool = []
    seen = set()
    
    for v in vars_:
        try:
            # 강화된 문서에서 검색
            temp_vs = FAISS.from_documents(enhanced_docs, embeddings)
            temp_retriever = temp_vs.as_retriever(search_kwargs={"k": 20})
            res = temp_retriever.invoke(v)
        except Exception:
            res = []
        
        for d in res:
            key = hash(d.page_content[:256])
            if key not in seen:
                seen.add(key)
                pool.append(d)
        
        if len(pool) >= top_k * 3:
            break
    
    if not pool:
        return "관련 문서를 찾을 수 없습니다.", []
    
    # 강화된 재랭킹
    final_docs = enhanced_precision_rerank(pool, query, top_k=top_k)
    
    # 컨텍스트 포맷팅
    ctx = format_context(final_docs)
    
    # 강화된 프롬프트로 LLM 호출
    chain = enhanced_prompt_template | llm | parser
    output = chain.invoke({"question": query, "context": ctx})
    
    return output, final_docs
  
# 강화된 시스템 테스트
print("🧪 강화된 시스템 테스트")
print("="*80)

test_query = "PNS의 purchaseState 값은 무엇인가요?"
print(f"Q: {test_query}")

enhanced_answer_result, enhanced_sources = enhanced_answer(test_query, top_k=8)

print(f"\n[Enhanced Answer]\n{enhanced_answer_result}")
print(f"\n[Enhanced Top sources]")
for i, d in enumerate(enhanced_sources, 1):
    md = d.metadata or {}
    doc_type = md.get('type', 'unknown')
    field = md.get('field', '')
    priority = md.get('priority', '')
    path = " > ".join(md.get('full_path', [])) if isinstance(md.get('full_path'), list) else ""
    print(f"- {i}) type={doc_type} field={field} priority={priority}")
    print(f"     path={path}")
    print(f"     content_preview={d.page_content[:100]}...")
    print()

📊 문서 통계: 섹션=495, 표=518, 필드정의=34, 총=1047
✅ 강화된 문서 세트가 준비되었습니다!
🧪 강화된 시스템 테스트
Q: PNS의 purchaseState 값은 무엇인가요?
🎯 필드값 쿼리 감지: PNS -> purchaseState
�� 상위 3개 문서 점수:
  1. 점수:52 | 타입:section | 필드:None
  2. 점수:51 | 타입:section | 필드:None
  3. 점수:48 | 타입:section | 필드:None

[Enhanced Answer]
제공된 컨텍스트에서 `PNS`의 `purchaseState` 값에 대한 구체적인 열거 사항은 명시되어 있지 않습니다. 주어진 정보는 `PurchaseData.PurchaseState` 열거형에서 구매 상태를 나타내는 값 중 하나를 반환한다는 점만 언급하고 있으며, 실제 값들의 목록은 포함되어 있지 않습니다. 따라서, 정확한 `purchaseState` 값들을 나열하는 것은 불가능합니다.

만약 `PurchaseData.PurchaseState` 열거형에 대한 일반적인 결제 상태 값들을 참고로 한다면, 일반적인 결제 상태는 다음과 같은 것들이 있을 수 있지만, 이는 문서에 명시된 정보만으로는 추정일 뿐입니다:

- 구매 완료: `PURCHASE_COMPLETE`
- 결제 취소: `PURCHASE_CANCELLED`
- 결제 중: `PURCHASE_IN_PROGRESS`
- 결제 실패: `PURCHASE_FAILED`

정확한 값들은 `PurchaseData.PurchaseState` 열거형 정의를 직접 참조해야 합니다. 문서 내에서 명확한 값 목록을 찾을 수 없으므로, 정확한 답변을 제공하기 어렵습니다. 정확한 값을 확인하려면 관련 API 문서나 `PurchaseData.PurchaseState` 정의를 직접 확인해야 합니다.

[Enhanced Top sources]
- 1) type=section field= priority=
     path=07. PNS(Payme